# Memory Agent Tool Test

This notebook demonstrates how to use the `MemoryAgentTool` with our KeyValueMemory implementation to create a single agent that can read from and write to memory.

The `MemoryAgentTool` class provides:
1. **Memory integration** for agents by extending BaseTool
2. **Pre-processing** via reader callbacks that fetch relevant context from memory before agent execution
3. **Post-processing** via parser callbacks that extract and store information from agent outputs after execution

In [1]:
import asyncio
import json
import logging
import re
from typing import Dict, Any, List, Optional

# Set up logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Reduce noise from autogen
logging.getLogger('autogen_core').setLevel(logging.WARNING)

In [2]:
# Import our KeyValueMemory and MemoryAgentTool
from memory import KeyValueMemory
from memory_agent_tool import MemoryAgentTool, MemoryAgentToolArgs

# Import necessary AutoGen components
from autogen_core import CancellationToken
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import OpenAIChatCompletionClient

## 1. Set up our memory store and model client

In [3]:
# Initialize the shared memory store
memory = KeyValueMemory(name="text_to_sql_memory")

# Set up the model client - replace with your specific model
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",  # or other appropriate model
    temperature=0.1,
    timeout=120
)

## 2. Define memory callback functions for our agent

The agent will have custom reader and parser functions that define how it interacts with memory.

In [4]:
# Schema Selector Agent memory callbacks
async def schema_selector_reader(memory, task, cancellation_token):
    """Read relevant info for the schema selector agent."""
    # For schema selection, we might want to read previously selected schemas
    # for similar queries to maintain consistency
    query_history_json = await memory.get("query_history")
    context = {}
    
    if query_history_json:
        try:
            query_history = json.loads(query_history_json)
            context["query_history"] = query_history
        except json.JSONDecodeError:
            logging.error("Failed to parse query history JSON")
        
    # We might also want to read any schema preferences
    schema_preferences_json = await memory.get("schema_preferences")
    if schema_preferences_json:
        try:
            schema_preferences = json.loads(schema_preferences_json)
            context["schema_preferences"] = schema_preferences
        except json.JSONDecodeError:
            logging.error("Failed to parse schema preferences JSON")
    
    return context

async def schema_selector_parser(memory, task, result, cancellation_token):
    """Parse and store schema selection results."""
    if result.messages:
        last_message = result.messages[-1].content
        print(f"Parsing message content: {last_message[:100]}...")  # Debug output
        
        # Look for database schema in XML format
        schema_match = re.search(r'<database_schema>.*?</database_schema>', last_message, re.DOTALL)
        if schema_match:
            schema_str = schema_match.group()
            await memory.set("current_schema", schema_str)
            print(f"Stored schema in memory: {schema_str[:100]}...")
        else:
            print("No database schema found in response")
            
            # As a fallback, try to extract from task if not found in response
            task_obj = json.loads(task)
            if "full_schema" in task_obj and task_obj["full_schema"]:
                await memory.set("current_schema", task_obj["full_schema"])
                print("Used schema from task as fallback")
            
        # Try to extract database ID from task
        try:
            task_obj = json.loads(task)
            if "db_id" in task_obj:
                db_id = task_obj["db_id"]
                await memory.set("current_db_id", db_id)
                print(f"Stored current_db_id: {db_id}")
        except json.JSONDecodeError:
            print("Could not parse task as JSON")
            
        # Store the task and response for history
        query_history_json = await memory.get("query_history")
        
        # Initialize or update query history
        query_history = []
        if query_history_json:
            try:
                query_history = json.loads(query_history_json)
            except json.JSONDecodeError:
                logging.error("Failed to parse existing query history, creating new one")
                
        # Parse the task
        query = ""
        try:
            task_obj = json.loads(task)
            query = task_obj.get("query", task)
        except json.JSONDecodeError:
            query = task
        
        # Add to history
        query_history.append({"query": query, "role": "selector"})
        
        # Store updated history as JSON string
        await memory.set("query_history", json.dumps(query_history))
        print(f"Updated query history with query: {query}")

## 3. Create our Agents with System Messages

In [5]:
# Define system messages for each agent
SCHEMA_SELECTOR_SYSTEM_MESSAGE = """
You are a database schema selector agent. Your role is to:
1. Analyze the natural language query
2. Extract relevant schema parts from the database
3. Return the selected schema in XML format wrapped in <database_schema> tags

Be precise and focus only on tables and columns directly related to the query.

Example output format:
<database_schema>
  <table name="customers">
    <column name="customer_id" type="INTEGER" primary_key="true" />
    <column name="name" type="TEXT" />
  </table>
  <table name="orders">
    <column name="order_id" type="INTEGER" primary_key="true" />
    <column name="customer_id" type="INTEGER" foreign_key="customers.customer_id" />
  </table>
</database_schema>

ALWAYS return your answer with the schema wrapped in <database_schema> tags.
"""

# Create the agents
schema_selector = AssistantAgent(
    name="schema_selector",
    system_message=SCHEMA_SELECTOR_SYSTEM_MESSAGE,
    model_client=model_client,
    description="Selects relevant parts of the database schema for a query"
)

# Wrap each agent with memory capabilities
schema_selector_tool = MemoryAgentTool(
    agent=schema_selector,
    memory=memory,
    reader_callback=schema_selector_reader,
    parser_callback=schema_selector_parser
)

## 4. Set up Sample Database Schema

We'll create a sample database schema to test the schema selector agent.

In [6]:
# Reset memory for a fresh run
await memory.clear()

# Set preferences that will be used by the schema selector
schema_preferences = {
    "max_tables": 5,
    "include_foreign_keys": True,
    "include_examples": True
}
await memory.set("schema_preferences", json.dumps(schema_preferences))

# Initialize empty query history
await memory.set("query_history", json.dumps([]))

print("Memory initialized with schema preferences")

2025-05-21 17:44:28,562 - root - INFO - [KeyValueMemory] Memory cleared.


Memory initialized with schema preferences


In [7]:
# Set up a sample database schema
full_schema = """
<database_schema>
  <table name="customers">
    <column name="customer_id" type="INTEGER" primary_key="true" />
    <column name="name" type="TEXT" />
    <column name="email" type="TEXT" />
    <column name="join_date" type="DATE" />
  </table>
  <table name="orders">
    <column name="order_id" type="INTEGER" primary_key="true" />
    <column name="customer_id" type="INTEGER" foreign_key="customers.customer_id" />
    <column name="order_date" type="DATE" />
    <column name="total_amount" type="DECIMAL" />
  </table>
  <table name="products">
    <column name="product_id" type="INTEGER" primary_key="true" />
    <column name="name" type="TEXT" />
    <column name="price" type="DECIMAL" />
    <column name="category" type="TEXT" />
  </table>
  <table name="order_items">
    <column name="item_id" type="INTEGER" primary_key="true" />
    <column name="order_id" type="INTEGER" foreign_key="orders.order_id" />
    <column name="product_id" type="INTEGER" foreign_key="products.product_id" />
    <column name="quantity" type="INTEGER" />
    <column name="price" type="DECIMAL" />
  </table>
  <table name="inventory">
    <column name="inventory_id" type="INTEGER" primary_key="true" />
    <column name="product_id" type="INTEGER" foreign_key="products.product_id" />
    <column name="quantity" type="INTEGER" />
    <column name="warehouse" type="TEXT" />
  </table>
</database_schema>
"""

# Store the full schema for the agent to read
await memory.set("full_database_schema", full_schema)
print("Full database schema stored in memory")

Full database schema stored in memory


## 6. Run the Schema Selector Agent for a Query

Let's run our agent with a sample query to see how it uses memory.

In [8]:
# Define a query and run the schema selector agent
query1 = "Find all customers who have placed orders with a total amount greater than $100"

# Create a task with the full schema and query
task1 = json.dumps({
    "query": query1,
    "db_id": "ecommerce",
    "full_schema": await memory.get("full_database_schema")
})

# Create a cancellation token
cancellation_token = CancellationToken()

# Create the proper arguments object
args = MemoryAgentToolArgs(task=task1)

# Run the agent
result1 = await schema_selector_tool.run(
    args=args,
    cancellation_token=cancellation_token
)

# Display result
print(f"\nAgent Response:\n{result1.messages[-1].content}")

2025-05-21 17:44:29,798 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Parsing message content: <database_schema>
  <table name="customers">
    <column name="customer_id" type="INTEGER" primary_k...
Stored schema in memory: <database_schema>
  <table name="customers">
    <column name="customer_id" type="INTEGER" primary_k...
Stored current_db_id: ecommerce
Updated query history with query: Find all customers who have placed orders with a total amount greater than $100

Agent Response:
<database_schema>
  <table name="customers">
    <column name="customer_id" type="INTEGER" primary_key="true" />
    <column name="name" type="TEXT" />
  </table>
  <table name="orders">
    <column name="order_id" type="INTEGER" primary_key="true" />
    <column name="customer_id" type="INTEGER" foreign_key="customers.customer_id" />
    <column name="total_amount" type="DECIMAL" />
  </table>
</database_schema>


## 7. Run a Second Query to Demonstrate Memory Continuity

Let's run a second query that's related. The agent should now have access to the previous query results through memory.

In [9]:
# Define a second query
query2 = "List all products ordered by customers with their total quantities"

# Create a task with the full schema and query
task2 = json.dumps({
    "query": query2,
    "db_id": "ecommerce",
    "full_schema": await memory.get("full_database_schema")
})

# Create the proper arguments object
args = MemoryAgentToolArgs(task=task2)

# Run the agent again
result2 = await schema_selector_tool.run(
    args=args,
    cancellation_token=cancellation_token
)

# Display result
print(f"\nAgent Response:\n{result2.messages[-1].content}")

2025-05-21 17:44:31,431 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Parsing message content: <database_schema>
  <table name="products">
    <column name="product_id" type="INTEGER" primary_key...
Stored schema in memory: <database_schema>
  <table name="products">
    <column name="product_id" type="INTEGER" primary_key...
Stored current_db_id: ecommerce
Updated query history with query: List all products ordered by customers with their total quantities

Agent Response:
<database_schema>
  <table name="products">
    <column name="product_id" type="INTEGER" primary_key="true" />
    <column name="name" type="TEXT" />
  </table>
  <table name="order_items">
    <column name="item_id" type="INTEGER" primary_key="true" />
    <column name="order_id" type="INTEGER" foreign_key="orders.order_id" />
    <column name="product_id" type="INTEGER" foreign_key="products.product_id" />
    <column name="quantity" type="INTEGER" />
  </table>
</database_schema>


## 8. Check Memory Contents

Let's examine what's stored in memory after our agent runs.

In [10]:
# Check what's stored in memory
current_schema = await memory.get("current_schema")
print(f"Current schema:\n{current_schema}\n")

db_id = await memory.get("current_db_id")
print(f"Current DB ID: {db_id}\n")

# Get query history
query_history_json = await memory.get("query_history")
query_history = json.loads(query_history_json)
print("Query History:")
for i, entry in enumerate(query_history):
    print(f"\nEntry {i+1}:")
    print(f"Query: {entry['query']}")
    print(f"Role: {entry['role']}")

Current schema:
<database_schema>
  <table name="products">
    <column name="product_id" type="INTEGER" primary_key="true" />
    <column name="name" type="TEXT" />
  </table>
  <table name="order_items">
    <column name="item_id" type="INTEGER" primary_key="true" />
    <column name="order_id" type="INTEGER" foreign_key="orders.order_id" />
    <column name="product_id" type="INTEGER" foreign_key="products.product_id" />
    <column name="quantity" type="INTEGER" />
  </table>
</database_schema>

Current DB ID: ecommerce

Query History:

Entry 1:
Query: Find all customers who have placed orders with a total amount greater than $100
Role: selector

Entry 2:
Query: List all products ordered by customers with their total quantities
Role: selector


## 9. Conclusion

This notebook demonstrates how the `MemoryAgentTool` allows a single schema selector agent to maintain context across multiple queries using memory. The key features demonstrated include:

1. Reading context from memory before agent execution
2. Parsing and storing results after agent execution
3. Maintaining history and state across multiple queries
4. Formatting memory context for clear communication with the agent

The schema selector agent can now maintain context about previously selected schemas, database preferences, and query history, which helps in making more consistent schema selections across related queries.